
<p>Задание заключается в том, чтобы написать код, который требуется в ячейках ###ваш код здесь</p>


In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import log

%matplotlib inline


<h3 id="Вероятности">Вероятности<a class="anchor-link" href="#Вероятности">¶</a></h3>



<h3 id="Метод-максимального-правдоподобия">Метод максимального правдоподобия<a class="anchor-link" href="#Метод-максимального-правдоподобия">¶</a></h3><p>А теперь допустим, что мы не знаем какая из граней утяжелённая. Всё, что у нас есть - это 1000 бросков этой кости. Как понять, какую грань утяжелили? Можно применить метод максимального правдоподобия (далее ММП). В данном случае, это стрелять из пушки по воробьям, ведь можно просто посмотреть, какая грань чаще всего выпала. Но зато очень наглядно.</p>



<p>У нашего замысловатого распределения $p$ есть два параметра: та грань, которую утяжелили, и кол-во граней. Кол-во граней известно заранее, поэтому его трогать не будем. Назовём грань, которую утяжелили - $\theta$. По формуле получаем оценку максимального правдоподобия для этого параметра $\theta_{ОМП}$:
$$\theta_{ОМП}=argmax_\theta P(X|\theta)$$</p>



<p>$P(X|\theta)$ - вероятность нашей выборки при конкретном параметре $\theta$. Учитывая то, что все эксперименты были независимыми, то можно её расписать как произведение вероятностей: $$P(X|\theta)=\prod_{i=0}^{N-1}P(\xi=x_i|\theta)$$
$\xi$ - это случайная величина броска игральной кости, а $x_i$ - результаты i-го броска (в нашем случае это элементы массива results).</p>



<p>$P(\xi=x_i|\theta) = 2/7$, если $x_i=\theta$, и $1/7$ в других случаях. Переберите всевозможные параметры $\theta$, выведите все вероятности и найдите самую большую.</p>



<p>Совет: используйте не $P(X|\theta)$, а возьмите логарифм: $$log(P(X|\theta))=\sum_{i=0}^{N-1}log(P(\xi=x_i|\theta))$$ Т.к. при умножении большого кол-ва чисел меньше 1 результат будет очень маленьким, и компьютер выставит просто 0.</p>


In [42]:
# свойства log: log a (b*c) = log a (b) + log a (c)

In [43]:
thetas = [1,2,3,4,5,6]
for theta in thetas:
    ### ваш код здесь
    summ = 0.
    for val in results:
        summ += np.log(2 / 7 if val == theta else 1 / 7)
    print(theta, summ)

1 -1843.3243663324674
2 -1851.642132499185
3 -1841.2449247907866
4 -1860.653045846469
5 -1854.4147212214273
6 -1731.034523081743



<p>Давайте посмотрим на условные распределения на реальных данных. Датасет для этой задачи будет из игры World of Warcraft. В ней есть "Поля боя" (Battlegrounds), на которой сражаются две фракции: Альянс(Alliance) и Орда(Horde). Каждую фракцию представляют игроки, которые могут быть различными классами (колонка Class). Битвы происходили несколько раз на разных полях боя, идентификатор одного боя - это колонка "Code".  В колонках Win, Lose показано, кто победил.</p>


In [3]:
data = pd.read_csv('wowbgs2.csv')
data.head()

,Battleground,Code,Faction,Class,KB,D,HK,DD,HD,Honor,Win,Lose,Rol,BE
0,WG,WG1,Horde,Hunter,1,3,14,48155,6641,532,1.0,NaN,dps,NaN
1,WG,WG1,Horde,Death Knight,1,3,12,27025,7106,377,1.0,NaN,dps,NaN
2,WG,WG1,Alliance,Paladin,0,1,19,824,93879,252,NaN,1.0,heal,NaN
3,WG,WG1,Alliance,Paladin,1,2,25,7046,98599,274,NaN,1.0,heal,NaN
4,WG,WG1,Alliance,Rogue,2,3,23,65483,19629,268,NaN,1.0,dps,NaN



<p>Колонка DD показывает, сколько персонаж нанёс урона. Колонка HD - сколько персонаж вылечил здоровья.</p>



<h4 id="Задание:">Задание:<a class="anchor-link" href="#Задание:">¶</a></h4><p>На вход подаётся несколько значений урона для какого-то персонажа. Ваша задача с помощью теорема Байеса определить, к кому, скорее всего, принадлежат данные цифры.</p>
<ol>
<li>В данном случае $A_j$ - это класс игрока, а $B$ - его урон. Вам нужно найти $P(A_j|B)$</li>
<li>Когда будете считать $P(B|A_j)$ можете просто пройтись по датасету и найти кол-во всех значений, которые лежат на интервале (B - 1000, B + 1000)</li>
</ol>


In [4]:
data.columns.values

array(['Battleground', 'Code', 'Faction', 'Class', 'KB', 'D', 'HK', 'DD',
       'HD', 'Honor', 'Win', 'Lose', 'Rol', 'BE'], dtype=object)

In [85]:
def P_B_Ai(B,Aj,data):
    tes =data[(data['DD']<(B+1000)) & (data['DD']>(B-1000))]
    if len(tes)==0 or len(tes[tes['Class']==Aj])==0:
        return 1
    return len(tes[tes['Class']==Aj])*1000/len(tes) #numbers in future can be small so python thinks it's near to null

In [86]:
def P_Aj_B(Aj,B,data):
    delt = 1
    for i in set(data['Class']):
        delt*=P_B_Ai(B,i,data)
        var = data[data['Class']==i]
        delt*=var.shape[0]
    return (data[data['Class']==Aj].shape[0])*P_B_Ai(B,Aj,data)*1000/delt

In [88]:
B = 50000
max_ver = 0
class_j = -1
for i in set(data['Class']):
    P_j_b = P_Aj_B(i,B,data)
    if P_j_b>max_ver:
        max_ver = P_j_b
        class_j = i
print(class_j)

Warrior



<h3 id="Наивный-Байес">Наивный Байес<a class="anchor-link" href="#Наивный-Байес">¶</a></h3>



<p>Пусть у нас есть обучающая выборка $X=(x_1,...x_N)$, каждый объект имеет $m$ признаков $w_1,...w_m$. И пусть мы решаем задачу классификации, т.е. в качестве ответа у нас есть $c$ классов. Наивный байесовский классификатор решает задачу следующим образом:
$$classify(w_1,...,w_n) = argmax_c\{P(c)\prod_{i=1}^{m}P(w_i|c)\}$$</p>



<p>Датасет для этой задачи будет из игры World of Warcraft. В ней есть "Поля боя" (Battlegrounds), на которой сражаются две фракции: Альянс(Alliance) и Орда(Horde). Каждую фракцию представляют игроки, которые могут быть различными классами (колонка Class). Битвы происходили несколько раз на разных полях боя, идентификатор одного боя - это колонка "Code".  В колонках Win, Lose показано, кто победил.</p>



<p>Допустим, начинается битва. Мы знаем, кто в нашей команде. Наша цель, понять, кто победит. В качестве признаков мы будем использовать кол-во каждого класса в каждой из команд. Я модифицировал датасет для этой задачи. Каждая строка - это отдельная битва. Колонки - это кол-во классов за каждую из фракций. target: 1 - победила Орда, 0 - победил Альянс</p>


In [3]:
df = pd.read_csv("wowbgs2.csv", index_col=0)
df.head()

,Code,Faction,Class,KB,D,HK,DD,HD,Honor,Win,Lose,Rol,BE
Battleground,,,,,,,,,,,,,
WG,WG1,Horde,Hunter,1,3,14,48155,6641,532,1.0,NaN,dps,NaN
WG,WG1,Horde,Death Knight,1,3,12,27025,7106,377,1.0,NaN,dps,NaN
WG,WG1,Alliance,Paladin,0,1,19,824,93879,252,NaN,1.0,heal,NaN
WG,WG1,Alliance,Paladin,1,2,25,7046,98599,274,NaN,1.0,heal,NaN
WG,WG1,Alliance,Rogue,2,3,23,65483,19629,268,NaN,1.0,dps,NaN


In [4]:
df = df.drop('Lose',1)

In [5]:
df['Win'] = df['Win'].replace(np.NaN,0)
df['Win'] = df['Win'].replace(1.0,1)

In [6]:
def normalize(df,name,count = 1000):
    dd1 = df[name]
    print(name,' ',max(dd1)-min(dd1))
    dd = np.array([0]*len(dd1))
    
    for i in range(len(dd1)):
        dd[i]=  round((dd1[i]-min(dd1))/count)
    df[name] = dd
    return df

In [7]:
names = ['DD','HD','Honor']
for name in names:
    df = normalize(df,name,int((max(df[name])-min(df[name]))/100))

DD   246000
HD   325000
Honor   1217


In [14]:
nmaes = ['Code','Faction','Class','Rol']
for name in names:
    replaces = {list(set(df[name]))[i]: i for i in range(len(set(df[name])))}
    row = df[name]
    new_row = np.array([0]*len(row))
    for i in range(len(row)):
        new_row[i] = replaces[row[i]]
    print(new_row)
    df[name] = new_row

[20 11  0 ... 16 22  5]
[ 2  2 29 ...  3  6  6]
[43 30 20 ... 22 22 23]


In [15]:
df.head()

,Code,Faction,Class,KB,D,HK,DD,HD,Honor,Win,Rol,BE
Battleground,,,,,,,,,,,,
WG,WG1,Horde,Hunter,1,3,14,20,2,43,1.0,dps,NaN
WG,WG1,Horde,Death Knight,1,3,12,11,2,30,1.0,dps,NaN
WG,WG1,Alliance,Paladin,0,1,19,0,29,20,0.0,heal,NaN
WG,WG1,Alliance,Paladin,1,2,25,3,30,22,0.0,heal,NaN
WG,WG1,Alliance,Rogue,2,3,23,27,6,21,0.0,dps,NaN



<p>Большая часть классификатора написана. Нужно написать функцию predict.</p>


In [9]:
def make_dict(arr):
    dicta = {}
    for i in arr:
        if dicta.get(i)==None:
            dicta.update({i:1})
        else:
            dicta.update({i:dicta.get(i)+1})
    return dicta

In [10]:
from math import log
class NaiveBaies():    
    def fit(self, X, y):
        # Список классов
        for i in range(len(y)):
            y[i]=int(y[i])
        self.classes = np.unique(y)
        # Сохраним кол-во классов
        self.cls_counts = len(set(y))
        
        # Сохраним для каждого значения признака кол-во таких объектов (P(w_i|c)), чтобы потом было легче обучаться
        # self.feature_counts[cls][column_idx][i] - 1+ кол-во элементов в обучающей выборке, с классом cls, у которых
        # признак под номером column_idx равен i
        self.feature_counts = {}
        for cls in self.classes:
            cur_feature_counts = {}
            for j in range(X.shape[1]):
                # Добавим ко всем значениям единицу, чтобы не было нулей в вероятностях
                arr = X[y == cls][j]
                arr  = make_dict(arr)
                cur_feature_counts[j] = arr
            self.feature_counts[cls] = cur_feature_counts
        return self
        
    def predict(self, X_test):
        preds = []
        for i in range(len(X_test)):
            line = X_test[i]
            count = 0
            key = 0.0
            for cls in self.classes:
                ans = log(sum(self.feature_counts[cls]))
                for j in range(len(line)):
                    ans*= self.feature_counts.get(cls,1).get(j,1).get(line[j],1)
                if ans>=count:
                    count = ans
                    key = cls
            preds.append(key)
        return preds

In [11]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, random_state=42, shuffle=True)   


<p>Можно попробовать и логистическую регрессию.</p>


In [13]:
for train, test in kf.split(df):
    df_train = df.iloc[train]
    df_test = df.iloc[test]
    X_train, y_train = df_train.drop("Win", axis=1).values, df_train["Win"].values
    X_test, y_test = df_test.drop("Win", axis=1).values, df_test["Win"].values

    clf = NaiveBaies()
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    summ= 0
    for i in range (len(preds)):
        if preds[i]==y_test[i]:
            summ+=1
    print(summ/len(preds))

0.5111441307578009
0.45988112927191677
0.5297176820208024
0.5174721189591078


In [16]:
names = ['KB', 'D', 'HK', 'DD', 'HD', 'Honor', 'Win' ,'BE']
for name in names:
    df[name].fillna(int(df[name].mean()), inplace=True)

In [21]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

df = df.drop('Code',1)
df = df.drop('Faction',1)
df = df.drop('Class',1)
df = df.drop('Rol',1)

cross_val_score(LogisticRegression(), df.drop("Win", axis=1).values, df["Win"].values, scoring="accuracy", cv=4)

array([0.93387816, 0.91381872, 0.89598811, 0.8929368 ])